In [ ]:
# Cheng S et al. A pan-cancer single-cell transcriptional atlas of tumor infiltrating myeloid cells. Cell 2021 
# https://www.sciencedirect.com/science/article/pii/S0092867421000106
# https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE154763
# https://nbisweden.github.io/workshop-scRNAseq/

In [ ]:
#!mkdir data
#!mkdir write
#!mkdir figures

In [ ]:
import time; start = time.time()

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [ ]:
from matplotlib.pyplot import rc_context
import matplotlib.pyplot as plt
#import sc_toolbox.api as sct
import seaborn as sns
import anndata
import pandas as pd
import scanpy as sc
import numpy as np
sc.logging.print_version_and_date()

In [ ]:
sc.settings.set_figure_params(scanpy=True, dpi=150, dpi_save=150, color_map='coolwarm',facecolor='white',
                             format='png', transparent=False, frameon=False, vector_friendly=True, fontsize=14)
sc.settings.verbosity = 0             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()

#   =   =   =   =    =   =   =   =   1. Get Data   =   =   =   =    =   =   =   =   

In [ ]:
results_file  = "./write/pbmc3k.h5ad"          # the file that will store the analysis results
data_file     = "./data/GSE154763_UCEC_norm_expression.csv.gz"
metadata_file = "./data/GSE154763_UCEC_metadata.csv.gz"

In [ ]:
%%time
UCEC     = pd.read_csv(data_file, index_col=0)
cellinfo = pd.read_csv(metadata_file, index_col=0)
geneinfo = pd.DataFrame(UCEC.columns, index=UCEC.columns, columns=['genes_index'])

In [ ]:
%%time
adata = sc.AnnData(UCEC, obs=cellinfo, var = geneinfo)

In [ ]:
adata.var_names_make_unique()  # unnecessary if using `var_names='gene_ids'` in `sc.read_10x_mtx`

#   =   =   =   =    =   =   =   =   2. Create One Merged Object   =   =   =   =    =   =   =   =   

In [ ]:
# merge into one object.
#adata = adata.concatenate(adata1, adata2)

In [ ]:
adata

In [ ]:
adata.obs.head(3)

In [ ]:
adata.obs.patient.value_counts()

#   =   =   =   =    =   =   =   =   3. Quality Control   =   =   =   =    =   =   =   =   

In [ ]:
adata.var['mt']   = adata.var_names.str.startswith('MT-')           # annotate mitochondrial genes
adata.var['ribo'] = adata.var_names.str.startswith(("RPS","RPL"))   # annotate ribosomal genes
adata.var['hb']   = adata.var_names.str.contains(("^HB[^(P)]"))     # annotate hemoglobin genes.
adata.var.head(3)

In [ ]:
# Calculate quality control metrics.
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt','ribo','hb'], percent_top=None, log1p=False, inplace=True)

In [ ]:
mito_genes = adata.var_names.str.startswith('MT-')
# for each cell compute fraction of counts in mito genes vs. all genes
# the `.A1` is only necessary as X is sparse (to transform to a dense array after summing)
adata.obs['percent_mt2'] = np.sum(adata[:, mito_genes].X, axis=1) / np.sum(adata.X, axis=1)
# add the total counts per cell as observations-annotation to adata
adata.obs['n_counts'] = adata.X.sum(axis=1)

In [ ]:
adata

#   =   =   =   =    =   =   =   =   4. Plot Quality Control Metrics   =   =   =   =    =   =   =   =   

In [ ]:
with plt.rc_context():  # Use this to set figure params like size and dpi
    sc.pl.violin(adata, ['n_genes','n_genes_by_counts', 'total_counts','total_counts_mt', 'pct_counts_mt'],
                 size=3, jitter=0.35, multi_panel=True, show=False)
    plt.tight_layout()
    plt.savefig("./figures/01a.QC_ViolinPlots.png", bbox_inches="tight")

In [ ]:
sc.pl.violin(adata, ['n_genes','n_genes_by_counts'], size=3, jitter=0.4, groupby='patient', rotation= 45)

In [ ]:
fig,(ax1) = plt.subplots(1,1, figsize=(15,4))
ax1 = sns.histplot(adata.obs["total_counts"], bins=100, kde=False, element="step", fill=False, ax=ax1)
#plt.savefig("test.png", bbox_inches="tight")

In [ ]:
# https://stackoverflow.com/questions/67157280/using-seaborn-displot-i-am-unable-to-specify-the-hue-to-separate-three-categorie

In [ ]:
# = = = = = = = = 5. Filtering = = = = = = = =

In [ ]:
sc.pp.filter_cells(adata, min_genes = 200) # Filter cell outliers based on counts and numbers of genes expressed.
sc.pp.filter_genes(adata, min_cells = 3)   # Filter genes based on number of cells or counts.
print(adata.n_obs, adata.n_vars)

In [ ]:
# Preprocessing. Show those genes that yield the highest fraction of counts in each single cell, across all cells.
with plt.rc_context():  # Use this to set figure params like size and dpi
    sc.pl.highest_expr_genes(adata, n_top=15, show=False)
    plt.savefig("./figures/01a.HighestFractionCounts.png", bbox_inches="tight")

In [ ]:
# Remove cells that have too many mitochondrial genes expressed or too many total counts:
fig,(ax1,ax2,ax3,ax4) = plt.subplots(1,4, figsize=(15,4))
ax1 = sns.histplot(adata.obs["total_counts"], bins=50, kde=False, element="step", fill=False, ax=ax1)
ax2 = sc.pl.scatter(adata, x='total_counts', y='pct_counts_mt', color='batch',     show=False,  legend_loc='right margin',  ax=ax2)
ax3 = sc.pl.scatter(adata, x='total_counts', y='n_genes_by_counts', color='batch', show=False,  legend_loc='right margin',  ax=ax3)
plt.tight_layout()
plt.savefig("./figures/01c.QC.png", bbox_inches="tight")

In [ ]:
adata = adata[adata.obs['pct_counts_mt'] < 20, :]   # filter for percent mito
adata = adata[adata.obs['pct_counts_ribo'] > 5, :]   # filter for percent ribo > 0.05
print("Remaining cells %d"%adata.n_obs)

In [ ]:
malat1 = adata.var_names.str.startswith('MALAT1')
# we need to redefine the mito_genes since they were first 
# calculated on the full object before removing low expressed genes.
mito_genes = adata.var_names.str.startswith('MT-')
hb_genes = adata.var_names.str.contains('^HB[^(P)]')
remove = np.add(mito_genes, malat1)
remove = np.add(remove, hb_genes)
keep = np.invert(remove)
adata = adata[:,keep]
print(adata.n_obs, adata.n_vars)

In [ ]:
print('Total number of observations before cell filter: {:d}'.format(adata.n_obs))
adata = adata[adata.obs.n_genes_by_counts < 2500, :]
adata = adata[adata.obs.pct_counts_mt < 5, :]
print('Total number of observations after cell filter: {:d}'.format(adata.n_obs))

In [ ]:
sc.pp.normalize_total(adata, target_sum=1e4) # Normalize counts per cell.
sc.pp.log1p(adata)                           # Logarithmize the data.

In [ ]:
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)    # Identify highly-variable genes.
adata = adata[:, adata.var.highly_variable]                                      # Actually do the filtering

In [ ]:
with plt.rc_context():  # Use this to set figure params like size and dpi
    sc.pl.highly_variable_genes(adata, highly_variable_genes=True, log=False, show=False)
    plt.savefig("./figures/01d.HighlyVariableGenes.png", bbox_inches="tight", log=True)

In [ ]:
# adata.X = adata.obsm['raw'].copy()

In [ ]:
adata.raw = adata

In [ ]:
print('Total number of observations: {:d}'.format(adata.n_obs))
print('Number of genes after cell filter: {:d}'.format(adata.n_vars))

In [ ]:
print('Total number of observations: {:d}'.format(adata.n_obs))
print('Number of genes after cell filter: {:d}'.format(adata.n_vars))

In [ ]:
#  Regress out effects of total counts per cell and the % mitochondrial genes expressed.
sc.pp.regress_out(adata, ['total_counts', 'pct_counts_mt'])
sc.pp.scale(adata, max_value=10)   # Scale each gene to unit variance. Clip values exceeding standard deviation 10. 

In [ ]:
adata.shape

In [ ]:
adata

In [ ]:
# = = = = = = = = 6. Sample sex = = = = = = = =

In [41]:
annot = sc.queries.biomart_annotations("hsapiens",
        ["ensembl_gene_id", "external_gene_name", "start_position", "end_position", "chromosome_name"],
    ).set_index("external_gene_name")
#>>> adata.var[annot.columns] = annot

In [42]:
chrY_genes = adata.var_names.intersection(annot.index[annot.chromosome_name == "Y"])
chrY_genes

Index([], dtype='object')

In [44]:
adata.obs['percent_chrY'] = np.sum(
    adata[:, chrY_genes].X, axis=1) / np.sum(adata.X, axis=1) * 100

C:\Users\alber\AppData\Local\Temp/ipykernel_1016/441416070.py:1: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['percent_chrY'] = np.sum(


In [45]:
# color inputs must be from either .obs or .var, so add in XIST expression to obs.
adata.obs["XIST-counts"] = adata.X[:,adata.var_names.str.match('XIST')].toarray()

sc.pl.scatter(adata, x='XIST-counts', y='percent_chrY', color="sample")

AttributeError: 'numpy.ndarray' object has no attribute 'toarray'

In [ ]:
sc.pl.violin(adata, ["XIST-counts", "percent_chrY"], jitter=0.4, groupby = 'sample', rotation= 45)

In [ ]:
# = = = = = = = = 7. Calculate cell-cycle scores = = = = = = = =

In [ ]:
adata.write(results_file)   # Save the result.

In [ ]:
print("'01_QualityControl.v001' script run time:", f'{time.time()-start:.0f}', "seconds.")

In [ ]:
#   =   =   =   =    =   =   =   =   STOP   =   =   =   =    =   =   =   =   